## Introduction

**Key point**
> It is probably true that custom metaclasses mostly aren’t necessary.
>
> If it isn’t pretty obvious that a problem calls for them,
> then it will probably be cleaner and more readable if solved in a simpler way.

### Old-Style vs. New-Style Classes

In the Python realm, a class can be [one of two varieties](https://wiki.python.org/moin/NewClassVsClassicClass).

No official terminology has been decided on, so they are informally referred to as **old-style** and **new-style** classes.

> In `Python 3`, **all classes are new-style classes**.
> Thus, in Python 3 it is reasonable to refer to an object’s type and its class interchangeably.

#### Old-Style Classes (Python 2)

With old-style classes, class and type are not quite the same thing.

An instance of an old-style class is always implemented from a single built-in type called `instance`.

If `obj` is an instance of an old-style class, `obj.__class__` designates the class, 
but `type(obj)` is always `instance`. 

The following example is taken from Python 2.7:

```python
>>> class Foo:
...     pass
...
>>> x = Foo()
>>> x.__class__
<class __main__.Foo at 0x000000000535CC48>
>>> type(x)
<type 'instance'>
```

#### New-Style Classes (Python 2 and 3)

New-style classes **unify the concepts of class and type**.

If `obj` is an instance of a new-style class, `type(obj)` is the same as `obj.__class__`:

In [1]:
class Foo:
    pass

obj = Foo()

In [2]:
obj.__class__

__main__.Foo

In [3]:
type(obj)

__main__.Foo

In [5]:
obj.__class__ is type(obj)  # NOTE.

True

In [6]:
n = 5

In [7]:
d = { 'x' : 1, 'y' : 2 }

In [8]:
class Foo:
    pass

In [9]:
x = Foo()

In [10]:
# Always true.

for obj in (n, d, x):
    print(type(obj) is obj.__class__)

True
True
True


## Type and Class

> In Python 3, all classes are new-style classes. 
> Thus, in Python 3 it is reasonable to refer to an object’s type and its class interchangeably.

(Not so in Python 2, but I don't need Python 2.)

---

In Python, **everything is an object**.

> Classes are objects as well.

As a result, a *class* must have a *type*. 

What is the *type* of a *class*?

(It's `type`.)

In [11]:
class Foo:
    pass

In [12]:
x = Foo()

In [13]:
type(x)

__main__.Foo

In [14]:
type(Foo)  # Here.

type

In [15]:
# The type of the built-in classes you are familiar with is also type:

for t in int, float, dict, list, tuple:
    print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [16]:
# For that matter, the type of type is type as well (yes, really):

type(type)

type

> `type` is a **metaclass**, of which *classes* are *instances*.

Just as an ordinary object is an instance of a class, **any class in Python 3 is an instance of the type metaclass**.

In the above case:
* `x` is an instance of *class* `Foo`.
* `Foo` is an instance of the *type* metaclass.
* `type` is also an instance of the `type` *metaclass*, so **it is an instance of itself** 🤯.

![img](./class-chain.webp)

## Defining a Class Dynamically

The built-in `type()` function, when passed one argument, returns the type of an object.

For new-style classes, that is generally the same as the object’s `__class__` attribute:

In [17]:
type(3)

int

In [18]:
type(['foo', 'bar', 'baz'])

list

In [19]:
t = (1, 2, 3, 4, 5)
type(t)

tuple

In [21]:
class Foo:
    pass

type(Foo())

__main__.Foo

❗ You can also call `type()` with three arguments - `type(<name>, <bases>, <dct>)`:

* \<name\> specifies the class name. This becomes the `__name__` attribute of the class.
* \<bases\> specifies a tuple of the base classes from which the class inherits. This becomes the `__bases__` attribute of the class.
* \<dct\> specifies a namespace dictionary containing definitions for the class body. This becomes the `__dict__` attribute of the class.

> Calling `type()` in this manner **creates a new instance of the type metaclass**. 
>
> In other words, it **dynamically creates a new class**.

---

In each of the following examples, the top snippet defines a class dynamically with `type()`,
while the snippet below it defines the class the usual way, with the `class` statement.

In each case, the two snippets are functionally equivalent.

#### Example 1

In this first example, the `<bases>` and `<dct>` arguments passed to `type()` are both empty.

No inheritance from any parent class is specified, and nothing is initially placed in the namespace dictionary. 

This is the simplest class definition possible:

In [22]:
Foo = type('Foo', (), {})

x = Foo()
x

In [23]:
# Equivalently...

class Foo:
    pass

x = Foo()
x

#### Example 2

Here, `<bases>` is a tuple with a single element `Foo`, specifying the parent class that `Bar` inherits from.

An attribute, `attr`, is initially placed into the namespace dictionary:

In [25]:
Bar = type('Bar', (Foo,), dict(attr=100))

x = Bar()
x.attr  # type: ignore

100

In [26]:
x.__class__

__main__.Bar

In [27]:
x.__class__.__bases__

(__main__.Foo,)

In [28]:
# Equivalently...

class Bar(Foo):
    attr = 100

x = Bar()
x.attr

100

In [29]:
x.__class__

__main__.Bar

In [30]:
x.__class__.__bases__

(__main__.Foo,)

#### Example 3

This time, `<bases>` is again empty.

Two objects are placed into the namespace dictionary via the `<dct>` argument. 

The first is an attribute named `attr` and the second a *function* named `attr_val`, which becomes a method of the defined class:

In [31]:
Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x : x.attr
    }
)

x = Foo()

In [33]:
x.attr # type: ignore

100

In [35]:
x.attr_val()  # type: ignore  # Our method.

100

In [36]:
# Equivalently...

class Foo:
    attr = 100
    def attr_val(self):
        return self.attr

x = Foo()

In [37]:
x.attr

100

In [38]:
x.attr_val()

100

#### Example 4

Only very simple functions can be defined with `lambda` in Python.

In the following example, a slightly more complex function is defined externally
then assigned to `attr_val` in the namespace dictionary via the name `f`:

In [39]:
def f(obj):  # NOTE: `obj` here is `self`. 
    print('attr =', obj.attr)

Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': f
    }
)

x = Foo()

In [40]:
x.attr  # type: ignore

100

In [41]:
x.attr_val()  # type: ignore

attr = 100


In [43]:
# Equivalently...

def f(obj):
    print('attr =', obj.attr)

class Foo:
    attr = 100
    attr_val = f

x = Foo()

In [44]:
x.attr

100

In [45]:
x.attr_val()

attr = 100


## Custom Metaclasses

**The instance-creation workflow**

Consider
```python
class Foo:
    pass

f = Foo()
```

The expression `Foo()` creates a new *instance* of *class* `Foo`.

When the interpreter encounters `Foo()`, the following occurs:
* The `__call__()` method of `Foo`'s **parent class** is called. Since `Foo` is a standard new-style class, 
its parent class is the **type metaclass**, so `type`'s `__call__()` method is invoked.
* [❗] That `__call__()` method in turn invokes the following*:
    * `__new__()`
    * `__init__()`

\* If `Foo` does not define `__new__()` and `__init__()`, default methods are inherited from `Foo`'s ancestry.
But if `Foo` does define these methods, they override those from the ancestry,
which *allows for customized behavior when instantiating `Foo`*.

---

In the following, a custom method called `new()` is defined and assigned as the `__new__()` method for `Foo`:

This modifies the instantiation behavior of class `Foo`:
each time an instance of `Foo` is created, by default it is initialized with an attribute called `attr`,
which has a value of `100`. 

Code like this would more usually appear in the `__init__()` method and not typically in `__new__()`. 
This example is contrived for demonstration purposes.

In [46]:
def new(cls):
    x = object.__new__(cls)  
    # ^ NOTE this line. Not really explained in the tutorial. 
    # Calling the __new__ method of the object base class (object is base of everything).
    x.attr = 100
    return x

Foo.__new__ = new

In [47]:
f = Foo()
f.attr

100

In [48]:
g = Foo()
g.attr

100

Now, as has already been reiterated, classes are objects too.

Suppose you wanted to similarly customize instantiation behavior when creating a class like `Foo`.

If you were to follow the pattern above, you’d again define a custom method and assign it as the `__new__()` method for
the class of which `Foo` is an instance.

`Foo` is an instance of the `type` metaclass, so the code looks something like this:

In [49]:
# Spoiler alert:  This doesn't work!

def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

Except, as you can see, you can’t reassign the `__new__()` method of the `type` metaclass. Python doesn’t allow it.

This is probably just as well. type is the metaclass from which all new-style classes are derived.
You really shouldn’t be mucking around with it anyway.
But then what recourse is there, if you want to customize instantiation of a class?

[❗] One possible solution is a **custom metaclass** [❗]

Essentially, instead of mucking around with the `type` metaclass,
you can *define your own metaclass, which derives from `type`*, and then you can muck around with that instead.

The first step is to define a metaclass that derives from `type`, as follows:

In [50]:
class Meta(type):  # Custom metaclass Meta that inherits the metaclass type.
    def __new__(cls, name, bases, dct):  # NOTE the full signature of __new__() here.
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100  # type: ignore
        return x

The definition header class `Meta(type)`: specifies that `Meta` derives from `type`.
Since `type` is a metaclass, that makes `Meta` a metaclass as well.

Note that a custom `__new__()` method has been defined for `Meta`.
It wasn’t possible to do that to the type metaclass directly. The `__new__()` method does the following:
* Delegates via `super()` to the `__new__()` method of the parent metaclass (`type`) to actually create a new class
* Assigns the custom attribute `attr` to the class, with a value of `100`
* Returns the newly created class

Now the other half of the voodoo: Define a new class `Foo` and **specify that its metaclass is the custom metaclass `Meta`**,
rather than the standard metaclass type.

This is done using the **`metaclass` keyword** in the class definition as follows:

In [52]:
class Foo(metaclass=Meta):
    pass

Foo.attr  # type: ignore

100

Voila!

`Foo` has picked up the `attr` attribute automatically from the `Meta` metaclass.

Of course, any other classes you define similarly will do likewise:

In [54]:
class Bar(metaclass=Meta):
    pass

class Qux(metaclass=Meta):
    pass

Bar.attr, Qux.attr  # type: ignore

(100, 100)

> In the same way that a class functions as a template for the creation of objects, a metaclass functions as a template for the creation of classes. 
>
> Metaclasses are sometimes referred to as *class factories*.

Compare the following two examples:

**Object factory (class)**

In [55]:
class Foo:
    def __init__(self):
        self.attr = 100

x = Foo()
x.attr

100

In [56]:
y = Foo()
y.attr

100

In [57]:
z = Foo()
z.attr

100

**Class factory (metaclass)**

⚠️ A very good example for understanding.

In [58]:
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100

class X(metaclass=Meta):
    pass

X.attr

100

In [59]:
class Y(metaclass=Meta):
    pass

Y.attr

100

In [60]:
class Z(metaclass=Meta):
    pass

Z.attr

100

## Is This Really Necessary?

As simple as the above class factory example is, it is the essence of how metaclasses work. They allow customization of class instantiation.

Still, this is a lot of fuss just to bestow the custom attribute attr on each newly created class. Do you really need a metaclass just for that?

In Python, there are at least a couple other ways in which effectively the same thing can be accomplished.

**Simple Inheritance:**

In [61]:
class Base:
    attr = 100


class X(Base):
    pass


class Y(Base):
    pass


class Z(Base):
    pass

In [62]:
X.attr

100

In [63]:
Y.attr

100

In [64]:
Z.attr

100

**Class Decorator:**

In [65]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class X:
    pass

@decorator
class Y:
    pass

@decorator
class Z:
    pass

In [66]:
X.attr

100

In [67]:
Y.attr

100

In [68]:
Z.attr

100